# Imports

In [ ]:
from __future__ import absolute_import
from __future__ import print_function
%matplotlib inline

import numpy as np
import pandas as pd
import re
import os
import csv
import sys
import random
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from scipy import interp
import scipy
import statsmodels.stats.api as sms

from itertools import cycle
from inspect import signature

from sklearn.metrics import (
    recall_score, classification_report, confusion_matrix, f1_score, auc, roc_curve,
    precision_recall_curve, average_precision_score, matthews_corrcoef,
    mean_squared_error, r2_score, mean_absolute_error
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression

from numpy.random import seed

import tensorflow as tf
from tensorflow import set_random_seed

import keras
from keras import backend as K
from keras.models import Sequential, Model, load_model
from keras.utils.np_utils import to_categorical
from keras.layers import (
    Bidirectional, LSTM, Lambda, Dropout, Dense, TimeDistributed, Masking,
    Activation, Input, Reshape, Embedding, GaussianNoise
)
from keras import initializers, optimizers, regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers.normalization import BatchNormalization

# eICU

In [ ]:
def get_eicu_result(min_time,skip_time):

    SEED_VALUE = 36
    seed(SEED_VALUE)
    np.random.seed(SEED_VALUE)
    random.seed(SEED_VALUE)
    os.environ['PYTHONHASHSEED']=str(SEED_VALUE)
    tf.set_random_seed(SEED_VALUE)

    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.7
    session = tf.Session(config=config)
    K.set_session(session)
    
    CV = True
    BATCH_SIZE = 128
    TASK = '%d_%d'%(min_time,skip_time)
    params = {'lr':0.000075,'hidden_units':128,'dropout':0.2,'epochs':50}
    #pos
    def pos_selection(df_pos,skip_time,min_time):
        posl = []
        posdf = pd.DataFrame(columns=df_pos.columns)
        all_matches = df_pos[df_pos['labelpt'] == df_pos['labelrec']]

        for i, p_id in enumerate(all_matches['patientunitstayid'].unique()):
            df_p_id = df_pos[df_pos['patientunitstayid'] == p_id]
            idx = all_matches[all_matches['patientunitstayid'] == p_id].index[0]
            t = df_pos.iloc[idx].itemoffset
            if t > (min_time + skip_time):
                posl.append(df_p_id[(df_p_id['itemoffset'] < (t-skip_time)) & (df_p_id['itemoffset'] > (t-(skip_time+min_time)))])

        posdf = pd.concat(posl,axis=0)
        return posdf

    #neg
    def neg_selection(df_neg,skip_time,min_time):
        negl = []
        negdf = pd.DataFrame(columns=df_neg.columns)

        for i, p_id in enumerate(df_neg['patientunitstayid'].unique()):
            df_p_id = df_neg[df_neg['patientunitstayid'] == p_id]
            t = df_p_id.iloc[-1].itemoffset
            if t > (min_time + skip_time):
                negl.append(df_p_id[(df_p_id['itemoffset'] < (t-skip_time)) &(df_p_id['itemoffset'] > (t-(min_time+skip_time)))])
        negdf = pd.concat(negl,axis=0)
        return negdf
    eicu_df  = pd.read_csv("eicu_df_all_24los_normed.csv")
    # ADD VASO
    eicu_df['vaso_dose'] = np.nan
    eicu_df['vaso_dose'] = eicu_df['rate_epinephrine']+eicu_df['rate_norepinephrine'] + eicu_df['rate_phenylephrine']/10 + eicu_df['rate_dopamine']/2
    eicu_df.drop(columns=['rate_epinephrine', 'rate_norepinephrine','rate_phenylephrine','rate_dopamine'],inplace=True)
    

    eicu_pos = eicu_df[eicu_df['CAM']==1]
    eicu_neg = eicu_df[eicu_df['CAM']==0]

    eicu_pos_filtered = pos_selection(eicu_pos,skip_time,min_time)
    eicu_neg_filtered = neg_selection(eicu_neg,skip_time,min_time)
    eicu_df_filtered = pd.concat([eicu_pos_filtered, eicu_neg_filtered],axis=0)


    trg = eicu_df_filtered.groupby('patientunitstayid')

    idtr = []
    train_np = []
    for idx, frame in trg:
        idtr.append(idx)
        train_np.append(frame)


    columns_ord = ['patientunitstayid', 'itemoffset', 'gender','sofa', 'sofa_wo_gcs', 'age', 'admissionheight',
           'admissionweight', 'Heart Rate', 'O2 Saturation', 'glucose',
           'Temperature (C)', 'sodium', 'BUN', 'WBC x 1000', 'Hemoglobin',
           'Platelets', 'Potassium', 'Chloride', 'Bicarbonate', 'Creatinine',
            'vent_flag', 'vaso_dose', 'CAM']

    def reader_deli(df_list,verbose=1):
        X_noncat = []
        X_cat = []
        deli = []
        nrows = []
        ts = []
        PID = []
        nb_unit_stays = len(df_list)
        for i, df in enumerate(df_list):
            if verbose:
                sys.stdout.write('\rFeed StayID {0} of {1}...'.format(i+1, nb_unit_stays))
            dft = df
            dummy = pd.DataFrame(columns=columns_ord)
            for c in columns_ord:
                dummy[c] = dft[c]        
            dft = dummy
            narr = np.array(dft)
            pid = narr[0,0]
            x_cat    = narr[:,2:5]
            x_noncat = narr[:, 5:-1]
            labeldeli = narr[0, -1]
            time = narr[:,1]
            X_cat.append(x_cat)
            X_noncat.append(x_noncat)
            deli.append(labeldeli)
            ts.append(time)
            nrows.append(narr.shape[0])
            PID.append(pid)
        PID = np.array(PID)    
        X_cat = np.array(X_cat)
        X_noncat = np.array(X_noncat)
        deli = np.array(deli)
        ts= np.array(ts)
        return PID,X_cat,X_noncat,ts,nrows,deli
    
    PID_tr,X_cat_tr,X_num_tr,ts_tr,nrows_tr,y_tr = reader_deli(train_np)
    
    def pad(arr, max_len= min_time):
        tmp = np.zeros((max_len, arr.shape[1]))
        tmp[:arr.shape[0], :arr.shape[1]] = arr
        return tmp  
    def ret_numpy(X):
        X_list = []
        for n in X:
            X_list.append(pad(n))
        return np.array(X_list)
    
    X_cat_tr = ret_numpy(X_cat_tr)
    X_num_tr = ret_numpy(X_num_tr)

    X_train = np.concatenate([X_num_tr,X_cat_tr],axis=-1)
    y_train = y_tr

    
    def data_reader(X,y):
        return X,y
    
    def f1(y_true, y_pred):
        y_pred = K.round(y_pred)
        tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
        tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
        fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
        fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)
        p = tp / (tp + fp + K.epsilon())
        r = tp / (tp + fn + K.epsilon())
        f1 = 2*p*r / (p+r+K.epsilon())
        f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
        return K.mean(f1)
    
    def plot_acc(history):
        plt.figure(figsize=(10,6))
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'validation'], loc='upper left')
        return plt.show()
    def plot_loss(history):
        plt.figure(figsize=(10,6))
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'validation'], loc='upper left')
        return plt.show()


    
    def plot_cum_auc(x_y, models, savename='plot.png', x_label='False Positive Rate', y_label='True Positive Rate',auprc=False, thr=0):
    
        colors = ['b', 'r','g']
        for i, (x_y_tuple, m) in enumerate(zip(x_y, models)):
            plt.plot(x_y_tuple[0], x_y_tuple[1],label='{}: {:.4f}'.format(m,auc(x_y_tuple[0], x_y_tuple[1])), color=colors[i])
        if auprc:
            plt.plot([0, 1], [thr, thr], linestyle='--', lw=2, color='k',label="Random: {:.4f}".format(thr), alpha=.8)

        else:
            plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='k',label="Random: {:.4f}".format(0.5000), alpha=.8)

        plt.xlabel(x_label, fontsize=15)
        plt.ylabel(y_label, fontsize=15)
        plt.axhline(0, color='black')
        plt.axvline(0, color='black')

        if auprc:
            legend = plt.legend(loc="upper left", prop={'size': 10}, bbox_to_anchor=(0.6, 0.95))
        else:
            legend = plt.legend(loc="lower right", prop={'size': 10}, bbox_to_anchor=(1, 0.05))
            
        plt.savefig(savename,
                     dpi=400, facecolor='white', transparent=True, bbox_extra_artists=(legend,), bbox_inches='tight')
        plt.show()
        
        
    ###########################################################
    def compute_metrics(X_test,y_test,y_probs):

        fpr, tpr, thresholds = roc_curve(y_test, y_probs)
        specat90 = 1-fpr[tpr>=0.90][0]
        intrp = interp(np.linspace(0, 1, 100), fpr, tpr)
        intrp[0] = 0.0

        roc_auc = auc(fpr, tpr)

        TN,FP,FN,TP = confusion_matrix(y_test, y_probs.round()).ravel()
        PPV = TP/(TP+FP)
        NPV = TN/(TN+FN)


        mcc = matthews_corrcoef(y_test, y_probs.round())
        
        #####################################################
        cw = get_class_weights(X_test,y_test)
        indx_pos = np.where(y_test == 1)
        indx_neg = np.where(y_test == 0)
        sample_w = np.array(np.zeros(len(y_test)))
        sample_w[indx_pos[0]] = cw[1]
        sample_w[indx_neg[0]] = cw[0]
        
        #####################################################
        
        recall_single  = recall_score(y_test,y_probs.round())
        
        average_precision = average_precision_score(y_test, y_probs)
        precision, recall, thresholds = precision_recall_curve(y_test, y_probs)
        prs = interp(np.linspace(0, 1, 100), recall[::-1], precision[::-1])
        prs[0] = 1.0
        prs[-1] = 0.0
        auc_prc = auc(recall, precision)
        
        return {'specat90': specat90, 'intrp': intrp,
                'fpr': fpr,
                'tpr': tpr, 'auc': roc_auc,
                'ppv': PPV, 'npv': NPV,
                'auc_prc': auc_prc,
                'prc':precision,
                'prs':prs,
                'rec':recall,
                'mcc': mcc,
                'rec_single':recall_single}
    
    
    
    def avg_metrics(results):
    
        mean_fpr = np.linspace(0,1,100)
        mean_recall = np.linspace(0, 1, 100)

        mean_tpr = np.mean([results[k]['intrp'] for k in results], axis=0)
        mean_tpr[-1] = 1.0
        std_tpr = np.std([results[k]['intrp'] for k in results], axis=0)
        mean_auc = auc(mean_fpr, mean_tpr)
        std_auc = np.std([results[k]['auc'] for k in results])
        ppvs = np.mean([results[k]['ppv'] for k in results])
        npvs = np.mean([results[k]['npv'] for k in results])
        mccs = np.mean([results[k]['mcc'] for k in results])
        specat90 = np.mean([results[k]['specat90'] for k in results])

        mean_precision = np.mean([results[k]['prs'] for k in results], axis=0)
        mean_precision[-1] = 0.0
        #########################
        mean_precision[0] = 1.0
        ########################
        mean_auc_prc = auc(mean_recall, mean_precision)
        std_auc_prc = np.std([results[k]['auc_prc'] for k in results], axis=0)
        recall_single = np.mean([results[k]['rec_single'] for k in results])
        
        ######################CI#########################
        l_CI, h_CI = sms.DescrStatsW([results[k]['auc'] for k in results]).tconfint_mean()
        auprc_l_CI, auprc_h_CI = sms.DescrStatsW([results[k]['auc_prc'] for k in results]).tconfint_mean()
        p_l_CI, p_h_CI = sms.DescrStatsW([results[k]['ppv'] for k in results]).tconfint_mean()
        r_l_CI, r_h_CI = sms.DescrStatsW([results[k]['rec_single'] for k in results]).tconfint_mean()
        ######################CI#########################
        print("Mean AUC: {0:0.4f} +- STD{1:0.4f}".format(mean_auc,std_auc))
        print("AUPRC: {0:0.4f} +- STD{1:0.4f}".format(mean_auc_prc,std_auc_prc))
        
            ######################CI#########################
        print("L_CI: {0:0.4f}, H_CI: {1:0.4f}".format(l_CI, h_CI))
        print("AUPRC_L_CI: {0:0.4f}, AUPRC_H_CI: {1:0.4f}".format(auprc_l_CI, auprc_h_CI))
        print("P_L_CI: {0:0.4f}, P_H_CI: {1:0.4f}".format(p_l_CI, p_h_CI))
        print("R_L_CI: {0:0.4f}, R_H_CI: {1:0.4f}".format(r_l_CI, r_h_CI))
            ######################CI#########################
    
        print("PPV: {0:0.4f}".format(ppvs))
        print("NPV: {0:0.4f}".format(npvs))
        print("MCC: {0:0.4f}".format(mccs))
        print("Spec@90: {0:0.4f}".format(specat90))
        print("Recall: {0:0.4f}".format(recall_single))
        

        return {'mean_auc': mean_auc,
                'tpr': mean_tpr,
                'std_auc':std_auc,
                'std_tpr': std_tpr,
                'ppv':ppvs,
                'npv':npvs,
                'mean_auc_prc':mean_auc_prc,
                'std_auc_prc':std_auc_prc,
                'mean_prc':mean_precision,
                'mean_recall':mean_recall,
                'mcc':mccs,
                'spec@90':specat90,
                'rec_single': recall_single,
                'l_CI':l_CI,
                'h_CI':h_CI,
                
                'p_l_CI':p_l_CI,
                'p_h_CI':p_h_CI,
                
                'r_l_CI':r_l_CI,
                'r_h_CI':r_h_CI,
                
                'auprc_l_CI':auprc_l_CI,
                'auprc_h_CI':auprc_h_CI}

    def plot_auc(result):
        plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='red',
                 label='Random', alpha=.8)

        mean_fpr = np.linspace(0,1,100)
        mean_tpr = result['tpr']
        mean_auc = result['mean_auc']
        std_auc = result['std_auc']
        plt.plot(mean_fpr, mean_tpr, color='b',
                 label=r'Mean ROC (AUC = %0.4f $\pm$ %0.4f)' % (mean_auc, std_auc),
                 lw=2, alpha=.8)

        std_tpr = result['std_tpr']
        tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
        tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
        plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                         label=r'$\pm$ 1 std. dev.')

        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.show()

    if CV:
        X_train,y_train= data_reader(X_train,y_train)
        
    # ORIGINAL
    def build_model(X_train,params):
        adam = optimizers.Adam(lr=params['lr'],decay=1e-6)

        input1 = keras.layers.Input(shape=(X_train.shape[1], 18),name='inp1')

        input2 = keras.layers.Input(shape=(X_train.shape[1], 3),name='inp2')

        x2 = Embedding(50, 2,name='emb',mask_zero=True)(input2)
        x2 = Lambda(lambda x2: x2, output_shape=lambda s:s)(x2)
        x2 = Reshape((int(x2.shape[1]),int(x2.shape[2]*x2.shape[3])),name='reshape1')(x2)

        merge = keras.layers.Concatenate(axis=-1,name='concat')([input1, x2])
        mask  = Masking(mask_value=0.,name="maski")(merge)

        lstm1 = Bidirectional(LSTM(units=params['hidden_units'], name= "lstm1",
                                   kernel_initializer='glorot_normal',return_sequences=False))(mask)
        lstm1 = Dropout(params['dropout'])(lstm1)  
        out = Dense(1,activation="sigmoid")(lstm1) 

        model = keras.models.Model(inputs=[input1, input2], outputs=out)
        model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

        return model
    
    def run_models(X_train, y_train, X_test, model,params=params ,savepath=None):
        cw = get_class_weights(X_train, y_train)
        if model == "LSTM":
            clf = build_model(X_train,params)

            es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
            checkpoint = ModelCheckpoint(filepath=savepath, monitor='val_f1',save_weights_only=False,
                                 verbose=1, save_best_only=True, mode='max')
            history = clf.fit([X_train[:,:,:18],X_train[:,:,18:]], y_train, batch_size=BATCH_SIZE,#validation_split=0.2,
                                class_weight=cw, epochs=params['epochs'],verbose=1,shuffle=True)#,
            clf.save(savepath)
            clf = load_model(savepath, custom_objects={'f1': f1})

            probs = clf.predict([X_test[:,:,:18],X_test[:,:,18:]])

        else:
            X_train_base = np.reshape(X_train,(X_train.shape[0],-1))
            y_train_base =np.expand_dims(y_train,axis=1)            
            X_test_base = np.reshape(X_test,(X_test.shape[0],-1))

            if model == "LR":
                clf = LogisticRegression(random_state=0, solver='liblinear',
                                     class_weight=cw).fit(X_train_base, y_train_base)
            elif model == "RF":
                clf = RandomForestClassifier(n_estimators=300, max_depth=6,random_state=36, max_features=8,class_weight=cw)\
                .fit(X_train_base, y_train_base)
            else:
                print('Invalid model name')

            probs = clf.predict_proba(X_test_base)[:, 1] 

        return probs
    
    def model_cv(X,y,model):
        i = 1
        cv_scores = {}

        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED_VALUE)
        for train, test in kfold.split(X, y):
            print(f'Fold: {i}')
            X_tr = X[train]
            X_ts = X[test]
            y_tr = y[train]
            y_ts = y[test]
            savepath = "eicu_%s_%d_cv.hdf5"%(TASK,i)
            probs = run_models(X_tr, y_tr, X_ts, model,params,savepath)
            cv_scores[i] = compute_metrics(X_ts,y_ts, probs)
            i += 1        

        cum_scores = avg_metrics(cv_scores)
        plot_auc(cum_scores)
        return cum_scores
    
    models = ['LR','RF','LSTM']
    results = {}
    for m in models:
        print(f'**********Model: {m}*********')
        results[m] = model_cv(X_train, y_train, model=m)

                 
   
        file_name = '{}min_{}skip_model_{}_eicu_CI_pr.json'.format(min_time,skip_time,m)
        reported_results = []
        for r in ['mean_auc', 'ppv', 'npv', 'mcc', 'spec@90','mean_auc_prc','rec_single','l_CI','h_CI','p_l_CI','p_h_CI','r_l_CI','r_h_CI','auprc_l_CI','auprc_h_CI']:
            
            reported_results.append(np.round_(results[m][r],decimals=4))
            with open(file_name, 'w') as f:
                f.write(str(reported_results))
        No_pt = X_train.shape[0]
        pos =  y_train[y_train ==1].shape[0]
        neg =  y_train[y_train ==0].shape[0]
        with open(file_name, 'a') as f:
            
            f.write(str("Total no:"))
            f.write(str(No_pt))
            f.write(str("positive no:"))
            f.write(str(pos))
            f.write(str("negative no:"))
            f.write(str(neg))
            
        ###########################
    mean_fpr = np.linspace(0,1,100)
    fpr_tprs = [(mean_fpr, results[m]['tpr']) for m in models]
    plot_cum_auc(fpr_tprs, models, savename='{}_cv_eicu_CI_pr.png'.format(TASK),auprc=False)

    thr = sum(y_train)/len(y_train)
    
    mean_prc_rec = [(np.linspace(0,1,100), results[m]['mean_prc']) for m in models]
    plot_cum_auc(mean_prc_rec, models, savename='{}_cv_prc_eicu_CI_pr.png'.format(TASK), x_label='Recall', y_label='Precision',auprc=True,thr=thr)


In [ ]:
for m in [48,24,12]:#min time
    for s in [96,72,48,24,12]:#pred time
        get_eicu_result(m,s)

## Recall


In [ ]:
def get_eicu_result_recall(min_time,skip_time):

    SEED_VALUE = 36
    seed(SEED_VALUE)
    np.random.seed(SEED_VALUE)
    random.seed(SEED_VALUE)
    os.environ['PYTHONHASHSEED']=str(SEED_VALUE)
    tf.set_random_seed(SEED_VALUE)

    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.7
    session = tf.Session(config=config)
    K.set_session(session)
    
    CV = True
    BATCH_SIZE = 128
    TASK = '%d_%d'%(min_time,skip_time)
    params = {'lr':0.000075,'hidden_units':128,'dropout':0.2,'epochs':50}
    #pos
    def pos_selection(df_pos,skip_time,min_time):
        posl = []
        posdf = pd.DataFrame(columns=df_pos.columns)
        all_matches = df_pos[df_pos['labelpt'] == df_pos['labelrec']]

        for i, p_id in enumerate(all_matches['patientunitstayid'].unique()):
            df_p_id = df_pos[df_pos['patientunitstayid'] == p_id]
            idx = all_matches[all_matches['patientunitstayid'] == p_id].index[0]
            t = df_pos.iloc[idx].itemoffset
            if t > (min_time + skip_time):
                posl.append(df_p_id[(df_p_id['itemoffset'] < (t-skip_time)) & (df_p_id['itemoffset'] > (t-(skip_time+min_time)))])

        posdf = pd.concat(posl,axis=0)
        return posdf

    #neg
    def neg_selection(df_neg,skip_time,min_time):
        negl = []
        negdf = pd.DataFrame(columns=df_neg.columns)

        for i, p_id in enumerate(df_neg['patientunitstayid'].unique()):
            df_p_id = df_neg[df_neg['patientunitstayid'] == p_id]
            t = df_p_id.iloc[-1].itemoffset
            if t > (min_time + skip_time):
                negl.append(df_p_id[(df_p_id['itemoffset'] < (t-skip_time)) &(df_p_id['itemoffset'] > (t-(min_time+skip_time)))])
        negdf = pd.concat(negl,axis=0)
        return negdf
    eicu_df  = pd.read_csv("eicu_df_all_24los_normed.csv")
    eicu_pos = eicu_df[eicu_df['CAM']==1]
    eicu_neg = eicu_df[eicu_df['CAM']==0]

    eicu_pos_filtered = pos_selection(eicu_pos,skip_time,min_time)
    eicu_neg_filtered = neg_selection(eicu_neg,skip_time,min_time)
    eicu_df_filtered = pd.concat([eicu_pos_filtered, eicu_neg_filtered],axis=0)
    trg = eicu_df_filtered.groupby('patientunitstayid')

    idtr = []
    train_np = []
    for idx, frame in trg:
        idtr.append(idx)
        train_np.append(frame)


    columns_ord = ['patientunitstayid', 'itemoffset', 'gender','sofa', 'sofa_wo_gcs', 'age', 'admissionheight',
           'admissionweight', 'Heart Rate', 'O2 Saturation', 'glucose',
           'Temperature (C)', 'sodium', 'BUN', 'WBC x 1000', 'Hemoglobin',
           'Platelets', 'Potassium', 'Chloride', 'Bicarbonate', 'Creatinine',
            'vent_flag', 'rate_dopamine', 'rate_epinephrine',
           'rate_norepinephrine', 'rate_phenylephrine', 'CAM']

    def reader_deli(df_list,verbose=1):
        X_noncat = []
        X_cat = []
        deli = []
        nrows = []
        ts = []
        PID = []
        nb_unit_stays = len(df_list)
        for i, df in enumerate(df_list):
            if verbose:
                sys.stdout.write('\rFeed StayID {0} of {1}...'.format(i+1, nb_unit_stays))
            dft = df
            dummy = pd.DataFrame(columns=columns_ord)
            for c in columns_ord:
                dummy[c] = dft[c]        
            dft = dummy
            narr = np.array(dft)
            pid = narr[0,0]
            x_cat    = narr[:,2:5]
            x_noncat = narr[:, 5:-1]
            labeldeli = narr[0, -1]
            time = narr[:,1]
            X_cat.append(x_cat)
            X_noncat.append(x_noncat)
            deli.append(labeldeli)
            ts.append(time)
            nrows.append(narr.shape[0])
            PID.append(pid)
        PID = np.array(PID)    
        X_cat = np.array(X_cat)
        X_noncat = np.array(X_noncat)
        deli = np.array(deli)
        ts= np.array(ts)
        return PID,X_cat,X_noncat,ts,nrows,deli
    
    PID_tr,X_cat_tr,X_num_tr,ts_tr,nrows_tr,y_tr = reader_deli(train_np)
    
    def pad(arr, max_len= min_time):
        tmp = np.zeros((max_len, arr.shape[1]))
        tmp[:arr.shape[0], :arr.shape[1]] = arr
        return tmp  
    def ret_numpy(X):
        X_list = []
        for n in X:
            X_list.append(pad(n))
        return np.array(X_list)
    
    X_cat_tr = ret_numpy(X_cat_tr)
    X_num_tr = ret_numpy(X_num_tr)

    X_train = np.concatenate([X_num_tr,X_cat_tr],axis=-1)
    y_train = y_tr

    
    def data_reader(X,y):
        return X,y
    
    def f1(y_true, y_pred):
        y_pred = K.round(y_pred)
        tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
        tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
        fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
        fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)
        p = tp / (tp + fp + K.epsilon())
        r = tp / (tp + fn + K.epsilon())
        f1 = 2*p*r / (p+r+K.epsilon())
        f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
        return K.mean(f1)
    
    def plot_acc(history):
        plt.figure(figsize=(10,6))
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'validation'], loc='upper left')
        return plt.show()
    def plot_loss(history):
        plt.figure(figsize=(10,6))
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'validation'], loc='upper left')
        return plt.show()


    
    def plot_cum_auc(x_y, models, savename='plot.png', x_label='False Positive Rate', y_label='True Positive Rate',auprc=False, thr=0):
    
        colors = ['b', 'r','g']
        for i, (x_y_tuple, m) in enumerate(zip(x_y, models)):
            plt.plot(x_y_tuple[0], x_y_tuple[1],label='{}: {:.4f}'.format(m,auc(x_y_tuple[0], x_y_tuple[1])), color=colors[i])
        if auprc:
            plt.plot([0, 1], [thr, thr], linestyle='--', lw=2, color='k',label="Random: {:.4f}".format(thr), alpha=.8)

        else:
            plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='k',label="Random: {:.4f}".format(0.5000), alpha=.8)

        plt.xlabel(x_label, fontsize=15)
        plt.ylabel(y_label, fontsize=15)
        plt.axhline(0, color='black')
        plt.axvline(0, color='black')

        if auprc:
            legend = plt.legend(loc="upper left", prop={'size': 10}, bbox_to_anchor=(0.6, 0.95))
        else:
            legend = plt.legend(loc="lower right", prop={'size': 10}, bbox_to_anchor=(1, 0.05))
            
        plt.savefig(savename,
                     dpi=400, facecolor='white', transparent=True, bbox_extra_artists=(legend,), bbox_inches='tight')
        plt.show()
    
    
    ###############################################
    def get_class_weights_recall(data, label):
        neg = data[label==0].shape[0]
        pos = len(data) - neg
        weight_for_0 = (1 / neg)*((pos+neg))/2.0 
        weight_for_1 = (1 / pos)*((pos+neg))
        return {0:weight_for_0 , 1:(weight_for_1)*2}
        
    ###########################################################
    def compute_metrics(X_test,y_test,y_probs):

        fpr, tpr, thresholds = roc_curve(y_test, y_probs)
        specat90 = 1-fpr[tpr>=0.90][0]
        intrp = interp(np.linspace(0, 1, 100), fpr, tpr)
        intrp[0] = 0.0

        roc_auc = auc(fpr, tpr)

        TN,FP,FN,TP = confusion_matrix(y_test, y_probs.round()).ravel()
        PPV = TP/(TP+FP)
        NPV = TN/(TN+FN)


        mcc = matthews_corrcoef(y_test, y_probs.round())
        
        #####################################################
        cw = get_class_weights_recall(X_test,y_test)
#         import pdb;pdb.set_trace()
        indx_pos = np.where(y_test == 1)
        indx_neg = np.where(y_test == 0)
        sample_w = np.array(np.zeros(len(y_test)))
        sample_w[indx_pos[0]] = cw[1]
        sample_w[indx_neg[0]] = cw[0]
        
        #####################################################
        
        recall_single  = recall_score(y_test,y_probs.round())        
        average_precision = average_precision_score(y_test, y_probs)
        precision, recall, thresholds = precision_recall_curve(y_test, y_probs)
        prs = interp(np.linspace(0, 1, 100), recall[::-1], precision[::-1])
        prs[0] = 1.0
        prs[-1] = 0.0
        auc_prc = auc(recall, precision)

        return {'specat90': specat90, 'intrp': intrp,
                'fpr': fpr,
                'tpr': tpr, 'auc': roc_auc,
                'ppv': PPV, 'npv': NPV,
                'auc_prc': auc_prc,
                'prc':precision,
                'prs':prs,
                'rec':recall,
                'mcc': mcc,
                'rec_single':recall_single}


    def avg_metrics(results):
    
        mean_fpr = np.linspace(0,1,100)
        mean_recall = np.linspace(0, 1, 100)

        mean_tpr = np.mean([results[k]['intrp'] for k in results], axis=0)
        mean_tpr[-1] = 1.0
        std_tpr = np.std([results[k]['intrp'] for k in results], axis=0)
        mean_auc = auc(mean_fpr, mean_tpr)
        std_auc = np.std([results[k]['auc'] for k in results])
        ppvs = np.mean([results[k]['ppv'] for k in results])
        npvs = np.mean([results[k]['npv'] for k in results])
        mccs = np.mean([results[k]['mcc'] for k in results])
        specat90 = np.mean([results[k]['specat90'] for k in results])

        mean_precision = np.mean([results[k]['prs'] for k in results], axis=0)
        mean_precision[-1] = 0.0
        #########################
        mean_precision[0] = 1.0
        ########################
        mean_auc_prc = auc(mean_recall, mean_precision)
        std_auc_prc = np.std([results[k]['auc_prc'] for k in results], axis=0)
        
        recall_single = np.mean([results[k]['rec_single'] for k in results])
        
        ######################CI#########################
        l_CI, h_CI = sms.DescrStatsW([results[k]['auc'] for k in results]).tconfint_mean()
        auprc_l_CI, auprc_h_CI = sms.DescrStatsW([results[k]['auc_prc'] for k in results]).tconfint_mean()
        p_l_CI, p_h_CI = sms.DescrStatsW([results[k]['ppv'] for k in results]).tconfint_mean()
        r_l_CI, r_h_CI = sms.DescrStatsW([results[k]['rec_single'] for k in results]).tconfint_mean()
        ######################CI#########################
        print("Mean AUC: {0:0.4f} +- STD{1:0.4f}".format(mean_auc,std_auc))
        print("AUPRC: {0:0.4f} +- STD{1:0.4f}".format(mean_auc_prc,std_auc_prc))
        
            ######################CI#########################
        print("L_CI: {0:0.4f}, H_CI: {1:0.4f}".format(l_CI, h_CI))
        print("AUPRC_L_CI: {0:0.4f}, AUPRC_H_CI: {1:0.4f}".format(auprc_l_CI, auprc_h_CI))
        print("P_L_CI: {0:0.4f}, P_H_CI: {1:0.4f}".format(p_l_CI, p_h_CI))
        print("R_L_CI: {0:0.4f}, R_H_CI: {1:0.4f}".format(r_l_CI, r_h_CI))
            ######################CI#########################
    
        print("PPV: {0:0.4f}".format(ppvs))
        print("NPV: {0:0.4f}".format(npvs))
        print("MCC: {0:0.4f}".format(mccs))
        print("Spec@90: {0:0.4f}".format(specat90))
        print("Recall: {0:0.4f}".format(recall_single))
        

        return {'mean_auc': mean_auc,
                'tpr': mean_tpr,
                'std_auc':std_auc,
                'std_tpr': std_tpr,
                'ppv':ppvs,
                'npv':npvs,
                'mean_auc_prc':mean_auc_prc,
                'std_auc_prc':std_auc_prc,
                'mean_prc':mean_precision,
                'mean_recall':mean_recall,
                'mcc':mccs,
                'spec@90':specat90,
                'rec_single': recall_single,
                'l_CI':l_CI,
                'h_CI':h_CI,
                
                'p_l_CI':p_l_CI,
                'p_h_CI':p_h_CI,
                
                'r_l_CI':r_l_CI,
                'r_h_CI':r_h_CI,
                
                'auprc_l_CI':auprc_l_CI,
                'auprc_h_CI':auprc_h_CI}
    
    def plot_auc(result):
        plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='red',
                 label='Random', alpha=.8)

        mean_fpr = np.linspace(0,1,100)
        mean_tpr = result['tpr']
        mean_auc = result['mean_auc']
        std_auc = result['std_auc']
        plt.plot(mean_fpr, mean_tpr, color='b',
                 label=r'Mean ROC (AUC = %0.4f $\pm$ %0.4f)' % (mean_auc, std_auc),
                 lw=2, alpha=.8)

        std_tpr = result['std_tpr']
        tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
        tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
        plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                         label=r'$\pm$ 1 std. dev.')

        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.show()


    
    
    if CV:
        X_train,y_train= data_reader(X_train,y_train)
        
    # ORIGINAL
    def build_model(X_train,params):
        adam = optimizers.Adam(lr=params['lr'],decay=1e-6)

        input1 = keras.layers.Input(shape=(X_train.shape[1], 21),name='inp1')

        input2 = keras.layers.Input(shape=(X_train.shape[1], 3),name='inp2')

        x2 = Embedding(50, 2,name='emb',mask_zero=True)(input2)
        x2 = Lambda(lambda x2: x2, output_shape=lambda s:s)(x2)
        x2 = Reshape((int(x2.shape[1]),int(x2.shape[2]*x2.shape[3])),name='reshape1')(x2)

        merge = keras.layers.Concatenate(axis=-1,name='concat')([input1, x2])
        mask  = Masking(mask_value=0.,name="maski")(merge)

        lstm1 = Bidirectional(LSTM(units=params['hidden_units'], name= "lstm1",
                                   kernel_initializer='glorot_normal',return_sequences=False))(mask)
        lstm1 = Dropout(params['dropout'])(lstm1)  
        out = Dense(1,activation="sigmoid")(lstm1) 

        model = keras.models.Model(inputs=[input1, input2], outputs=out)
        model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

        return model
    
    def run_models(X_train, y_train, X_test, model,params=params ,savepath=None):
        cw = get_class_weights_recall(X_train, y_train)
        if model == "LSTM":
            clf = build_model(X_train,params)

            es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
            checkpoint = ModelCheckpoint(filepath=savepath, monitor='val_f1',save_weights_only=False,
                                 verbose=1, save_best_only=True, mode='max')
            history = clf.fit([X_train[:,:,:21],X_train[:,:,21:]], y_train, batch_size=BATCH_SIZE,#validation_split=0.2,
                                class_weight=cw, epochs=params['epochs'],verbose=1,shuffle=True)#,
            clf.save(savepath)
            clf = load_model(savepath, custom_objects={'f1': f1})

            probs = clf.predict([X_test[:,:,:21],X_test[:,:,21:]])

        else:
            X_train_base = np.reshape(X_train,(X_train.shape[0],-1))
            y_train_base =np.expand_dims(y_train,axis=1)            
            X_test_base = np.reshape(X_test,(X_test.shape[0],-1))

            if model == "LR":
                clf = LogisticRegression(random_state=0, solver='liblinear',
                                     class_weight=cw).fit(X_train_base, y_train_base)
            elif model == "RF":
                clf = RandomForestClassifier(n_estimators=300, max_depth=6,random_state=36, max_features=8,class_weight=cw)\
                .fit(X_train_base, y_train_base)
            else:
                print('Invalid model name')

            probs = clf.predict_proba(X_test_base)[:, 1] 

        return probs
    
    def model_cv(X,y,model):
        i = 1
        cv_scores = {}

        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED_VALUE)
        for train, test in kfold.split(X, y):
            print(f'Fold: {i}')
            X_tr = X[train]
            X_ts = X[test]
            y_tr = y[train]
            y_ts = y[test]
            savepath = "eicu_%s_%d_recall_cv_CI.hdf5"%(TASK,i)
            probs = run_models(X_tr, y_tr, X_ts, model,params,savepath)
            cv_scores[i] = compute_metrics(X_ts,y_ts, probs)
            i += 1        

        cum_scores = avg_metrics(cv_scores)
        plot_auc(cum_scores)
        return cum_scores
    
    models = ['LR','RF','LSTM']
    results = {}
    for m in models:
        print(f'**********Model: {m}*********')
        results[m] = model_cv(X_train, y_train, model=m)

                 
   
        file_name = '{}min_{}skip_model_{}_eicu_recall_CI_pr.json'.format(min_time,skip_time,m)
        reported_results = []
        
#         for r in ['mean_auc', 'ppv', 'npv', 'mcc', 'spec@90','mean_auc_prc','rec_single','l_CI','h_CI']:
        for r in ['mean_auc', 'ppv', 'npv', 'mcc', 'spec@90','mean_auc_prc','rec_single','l_CI','h_CI','p_l_CI','p_h_CI','r_l_CI','r_h_CI','auprc_l_CI','auprc_h_CI']:

            
            reported_results.append(np.round_(results[m][r],decimals=4))
            with open(file_name, 'w') as f:
                f.write(str(reported_results))
        No_pt = X_train.shape[0]
        pos =  y_train[y_train ==1].shape[0]
        neg =  y_train[y_train ==0].shape[0]
        with open(file_name, 'a') as f:
            f.write(str("Total no:"))
            f.write(str(No_pt))
            f.write(str("positive no:"))
            f.write(str(pos))
            f.write(str("negative no:"))
            f.write(str(neg))
        
    mean_fpr = np.linspace(0,1,100)
    fpr_tprs = [(mean_fpr, results[m]['tpr']) for m in models]
    plot_cum_auc(fpr_tprs, models, savename='{}_cv_recall_eicu_CI_pr.png'.format(TASK),auprc=False)

    thr = sum(y_train)/len(y_train)
    
    mean_prc_rec = [(np.linspace(0,1,100), results[m]['mean_prc']) for m in models]
    plot_cum_auc(mean_prc_rec, models, savename='{}_cv_prc_recall_eicu_CI_pr.png'.format(TASK), x_label='Recall', y_label='Precision',auprc=True,thr=thr)


In [ ]:
for m in [48,24,12]:#min time
    for s in [96,72,48,24,12]: #pred time
        get_eicu_result_recall(m,s)

In [ ]:
for m in [24]:#min time
    for s in [96,72,48,24,12]: #pred time
        get_eicu_result_recall(m,s)

# MIMIC

In [ ]:
def get_mimic_result(min_time,skip_time,high_recall= False):

    SEED_VALUE = 36
    seed(SEED_VALUE)
    np.random.seed(SEED_VALUE)
    random.seed(SEED_VALUE)
    os.environ['PYTHONHASHSEED']=str(SEED_VALUE)
    tf.set_random_seed(SEED_VALUE)

    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.7
    session = tf.Session(config=config)
    K.set_session(session)
    
    CV = True
    BATCH_SIZE = 128
    TASK = '%d_%d'%(min_time,skip_time)
    params = {'lr':0.000075,'hidden_units':128,'dropout':0.2,'epochs':50}
    #pos
    def pos_selection(df_pos,skip_time,min_time):
        posl = []
        posdf = pd.DataFrame(columns=df_pos.columns)
        all_matches = df_pos[df_pos['labelpt'] == df_pos['labelrec']]

        for i, p_id in enumerate(all_matches['patientunitstayid'].unique()):
            df_p_id = df_pos[df_pos['patientunitstayid'] == p_id]
            idx = all_matches[all_matches['patientunitstayid'] == p_id].index[0]
            t = df_pos.iloc[idx].itemoffset
            if t > (min_time + skip_time):
                posl.append(df_p_id[(df_p_id['itemoffset'] < (t-skip_time)) & (df_p_id['itemoffset'] > (t-(skip_time+min_time)))])

        posdf = pd.concat(posl,axis=0)
        return posdf

    #neg
    def neg_selection(df_neg,skip_time,min_time):
        negl = []
        negdf = pd.DataFrame(columns=df_neg.columns)

        for i, p_id in enumerate(df_neg['patientunitstayid'].unique()):
            df_p_id = df_neg[df_neg['patientunitstayid'] == p_id]
            t = df_p_id.iloc[-1].itemoffset
            if t > (min_time + skip_time):
                negl.append(df_p_id[(df_p_id['itemoffset'] < (t-skip_time)) &(df_p_id['itemoffset'] > (t-(min_time+skip_time)))])
        negdf = pd.concat(negl,axis=0)
        return negdf
    mimic_df = pd.read_csv("mimic_df_all_24los_normed.csv")

 # ADD VASO
    mimic_df['vaso_dose'] = np.nan
    mimic_df['vaso_dose'] = mimic_df['rate_epinephrine']+mimic_df['rate_norepinephrine'] + mimic_df['rate_phenylephrine']/10 + mimic_df['rate_dopamine']/2
    mimic_df.drop(columns=['rate_epinephrine', 'rate_norepinephrine','rate_phenylephrine','rate_dopamine'],inplace=True)
    mimic_pos = mimic_df[mimic_df['CAM']==1]
    mimic_neg = mimic_df[mimic_df['CAM']==0]
    mimic_pos_filtered = pos_selection(mimic_pos,skip_time,min_time)
    mimic_neg_filtered = neg_selection(mimic_neg,skip_time,min_time)
    mimic_df_filtered = pd.concat([mimic_pos_filtered, mimic_neg_filtered],axis=0)

    tsg  = mimic_df_filtered.groupby('patientunitstayid')

    idts = []
    test_np = []
    for idx, frame in tsg:
        idts.append(idx)
        test_np.append(frame)

    columns_ord = ['patientunitstayid', 'itemoffset', 'gender','sofa', 'sofa_wo_gcs', 'age', 'admissionheight',
           'admissionweight', 'Heart Rate', 'O2 Saturation', 'glucose',
           'Temperature (C)', 'sodium', 'BUN', 'WBC x 1000', 'Hemoglobin',
           'Platelets', 'Potassium', 'Chloride', 'Bicarbonate', 'Creatinine',
            'vent_flag', 'vaso_dose', 'CAM']

    def reader_deli(df_list,verbose=1):
        X_noncat = []
        X_cat = []
        deli = []
        nrows = []
        ts = []
        PID = []
        nb_unit_stays = len(df_list)
        for i, df in enumerate(df_list):
            if verbose:
                sys.stdout.write('\rFeed StayID {0} of {1}...'.format(i+1, nb_unit_stays))
            dft = df
            dummy = pd.DataFrame(columns=columns_ord)
            for c in columns_ord:
                dummy[c] = dft[c]        
            dft = dummy
            narr = np.array(dft)
            pid = narr[0,0]
            x_cat    = narr[:,2:5]
            x_noncat = narr[:, 5:-1]
            labeldeli = narr[0, -1]
            time = narr[:,1]
            X_cat.append(x_cat)
            X_noncat.append(x_noncat)
            deli.append(labeldeli)
            ts.append(time)
            nrows.append(narr.shape[0])
            PID.append(pid)
        PID = np.array(PID)    
        X_cat = np.array(X_cat)
        X_noncat = np.array(X_noncat)
        deli = np.array(deli)
        ts= np.array(ts)
        return PID,X_cat,X_noncat,ts,nrows,deli
    
    PID_ts,X_cat_ts,X_num_ts,ts_ts,nrows_ts,y_ts = reader_deli(test_np)
    
    def pad(arr, max_len= min_time):
        tmp = np.zeros((max_len, arr.shape[1]))
        tmp[:arr.shape[0], :arr.shape[1]] = arr
        return tmp  
    def ret_numpy(X):
        X_list = []
        for n in X:
            X_list.append(pad(n))
        return np.array(X_list)
    X_cat_ts = ret_numpy(X_cat_ts)
    X_num_ts = ret_numpy(X_num_ts)
    X_test  = np.concatenate([X_num_ts,X_cat_ts],axis=-1)
    y_test  = y_ts

    
    def data_reader(X,y):
        return X,y
    
    def f1(y_true, y_pred):
        y_pred = K.round(y_pred)
        tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
        tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
        fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
        fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)
        p = tp / (tp + fp + K.epsilon())
        r = tp / (tp + fn + K.epsilon())
        f1 = 2*p*r / (p+r+K.epsilon())
        f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
        return K.mean(f1)
    
    def plot_acc(history):
        plt.figure(figsize=(10,6))
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'validation'], loc='upper left')
        return plt.show()
    def plot_loss(history):
        plt.figure(figsize=(10,6))
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'validation'], loc='upper left')
        return plt.show()


    
    def plot_cum_auc(x_y, models, savename='plot.png', x_label='False Positive Rate', y_label='True Positive Rate',auprc=False, thr=0):
    
        colors = ['b', 'r','g']
        for i, (x_y_tuple, m) in enumerate(zip(x_y, models)):
            plt.plot(x_y_tuple[0], x_y_tuple[1],label='{}: {:.4f}'.format(m,auc(x_y_tuple[0], x_y_tuple[1])), color=colors[i])
        if auprc:
            plt.plot([0, 1], [thr, thr], linestyle='--', lw=2, color='k',label="Random: {:.4f}".format(thr), alpha=.8)

        else:
            plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='k',label="Random: {:.4f}".format(0.5000), alpha=.8)

        plt.xlabel(x_label, fontsize=15)
        plt.ylabel(y_label, fontsize=15)
        plt.axhline(0, color='black')
        plt.axvline(0, color='black')

        if auprc:
            legend = plt.legend(loc="upper left", prop={'size': 10}, bbox_to_anchor=(0.6, 0.95))
        else:
            legend = plt.legend(loc="lower right", prop={'size': 10}, bbox_to_anchor=(1, 0.05))
            
        plt.savefig(savename,
                     dpi=400, facecolor='white', transparent=True, bbox_extra_artists=(legend,), bbox_inches='tight')
        plt.show()
        
        
    ###########################################################
    def compute_metrics(X_test,y_test,y_probs):

        fpr, tpr, thresholds = roc_curve(y_test, y_probs)
        specat90 = 1-fpr[tpr>=0.90][0]
        intrp = interp(np.linspace(0, 1, 100), fpr, tpr)
        intrp[0] = 0.0
        roc_auc = auc(fpr, tpr)
        TN,FP,FN,TP = confusion_matrix(y_test, y_probs.round()).ravel()
        PPV = TP/(TP+FP)
        NPV = TN/(TN+FN)
        mcc = matthews_corrcoef(y_test, y_probs.round())
        #####################################################
        cw = get_class_weights(X_test,y_test)
        indx_pos = np.where(y_test == 1)
        indx_neg = np.where(y_test == 0)
        sample_w = np.array(np.zeros(len(y_test)))
        sample_w[indx_pos[0]] = cw[1]
        sample_w[indx_neg[0]] = cw[0]
        #####################################################
        
        recall_single  = recall_score(y_test,y_probs.round())
        
        average_precision = average_precision_score(y_test, y_probs)
        precision, recall, thresholds = precision_recall_curve(y_test, y_probs)
        prs = interp(np.linspace(0, 1, 100), recall[::-1], precision[::-1])
        prs[0] = 1.0
        prs[-1] = 0.0

        auc_prc = auc(recall, precision)

        return {'specat90': specat90, 'intrp': intrp,
                'fpr': fpr,
                'tpr': tpr, 'auc': roc_auc,
                'ppv': PPV, 'npv': NPV,
                'auc_prc': auc_prc,
                'prc':precision,
                'prs':prs,
                'rec':recall,
                'mcc': mcc,
                'rec_single':recall_single}


    def avg_metrics(results):
    
        mean_fpr = np.linspace(0,1,100)
        mean_recall = np.linspace(0, 1, 100)

        mean_tpr = np.mean([results[k]['intrp'] for k in results], axis=0)
        mean_tpr[-1] = 1.0
        std_tpr = np.std([results[k]['intrp'] for k in results], axis=0)
        mean_auc = auc(mean_fpr, mean_tpr)
        std_auc = np.std([results[k]['auc'] for k in results])
        ppvs = np.mean([results[k]['ppv'] for k in results])
        npvs = np.mean([results[k]['npv'] for k in results])
        mccs = np.mean([results[k]['mcc'] for k in results])
        specat90 = np.mean([results[k]['specat90'] for k in results])

        mean_precision = np.mean([results[k]['prs'] for k in results], axis=0)
        mean_precision[-1] = 0.0
        #########################
        mean_precision[0] = 1.0
        ########################
        mean_auc_prc = auc(mean_recall, mean_precision)
        std_auc_prc = np.std([results[k]['auc_prc'] for k in results], axis=0)
        
        recall_single = np.mean([results[k]['rec_single'] for k in results])
        
        
        ######################CI#########################
        l_CI, h_CI = sms.DescrStatsW([results[k]['auc'] for k in results]).tconfint_mean()
        auprc_l_CI, auprc_h_CI = sms.DescrStatsW([results[k]['auc_prc'] for k in results]).tconfint_mean()
        p_l_CI, p_h_CI = sms.DescrStatsW([results[k]['ppv'] for k in results]).tconfint_mean()
        r_l_CI, r_h_CI = sms.DescrStatsW([results[k]['rec_single'] for k in results]).tconfint_mean()
        ######################CI#########################
        print("Mean AUC: {0:0.4f} +- STD{1:0.4f}".format(mean_auc,std_auc))
        print("AUPRC: {0:0.4f} +- STD{1:0.4f}".format(mean_auc_prc,std_auc_prc))
        
        ######################CI#########################
        print("L_CI: {0:0.4f}, H_CI: {1:0.4f}".format(l_CI, h_CI))
        print("AUPRC_L_CI: {0:0.4f}, AUPRC_H_CI: {1:0.4f}".format(auprc_l_CI, auprc_h_CI))
        print("P_L_CI: {0:0.4f}, P_H_CI: {1:0.4f}".format(p_l_CI, p_h_CI))
        print("R_L_CI: {0:0.4f}, R_H_CI: {1:0.4f}".format(r_l_CI, r_h_CI))
        ######################CI#########################
    
        print("PPV: {0:0.4f}".format(ppvs))
        print("NPV: {0:0.4f}".format(npvs))
        print("MCC: {0:0.4f}".format(mccs))
        print("Spec@90: {0:0.4f}".format(specat90))
        print("Recall: {0:0.4f}".format(recall_single))
        

        return {'mean_auc': mean_auc,
                'tpr': mean_tpr,
                'std_auc':std_auc,
                'std_tpr': std_tpr,
                'ppv':ppvs,
                'npv':npvs,
                'mean_auc_prc':mean_auc_prc,
                'std_auc_prc':std_auc_prc,
                'mean_prc':mean_precision,
                'mean_recall':mean_recall,
                'mcc':mccs,
                'spec@90':specat90,
                'rec_single': recall_single,
                'l_CI':l_CI,
                'h_CI':h_CI,
                
                'p_l_CI':p_l_CI,
                'p_h_CI':p_h_CI,
                
                'r_l_CI':r_l_CI,
                'r_h_CI':r_h_CI,
                
                'auprc_l_CI':auprc_l_CI,
                'auprc_h_CI':auprc_h_CI}

    def plot_auc(result):
        plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='red',
                 label='Random', alpha=.8)

        mean_fpr = np.linspace(0,1,100)
        mean_tpr = result['tpr']
        mean_auc = result['mean_auc']
        std_auc = result['std_auc']
        plt.plot(mean_fpr, mean_tpr, color='b',
                 label=r'Mean ROC (AUC = %0.4f $\pm$ %0.4f)' % (mean_auc, std_auc),
                 lw=2, alpha=.8)

        std_tpr = result['std_tpr']
        tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
        tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
        plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                         label=r'$\pm$ 1 std. dev.')

        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.show()
        
    def get_class_weights(data, label):
        neg = data[label==0].shape[0]
        pos = len(data) - neg
        weight_for_0 = (1 / neg)*((pos+neg))/2.0 
        weight_for_1 = (1 / pos)*((pos+neg))
        if high_recall==False:
            return {0:weight_for_0 , 1:(weight_for_1)/2}
        elif high_recall:
            return {0:weight_for_0 , 1:(weight_for_1)}
        

    if CV:
        X_test,y_test= data_reader(X_test,y_test)
        
    # ORIGINAL
    def build_model(X_test,params):
        adam = optimizers.Adam(lr=params['lr'],decay=1e-6)

        input1 = keras.layers.Input(shape=(X_test.shape[1], 18),name='inp1')

        input2 = keras.layers.Input(shape=(X_test.shape[1], 3),name='inp2')

        x2 = Embedding(50, 2,name='emb',mask_zero=True)(input2)
        x2 = Lambda(lambda x2: x2, output_shape=lambda s:s)(x2)
        x2 = Reshape((int(x2.shape[1]),int(x2.shape[2]*x2.shape[3])),name='reshape1')(x2)

        merge = keras.layers.Concatenate(axis=-1,name='concat')([input1, x2])
        mask  = Masking(mask_value=0.,name="maski")(merge)

        lstm1 = Bidirectional(LSTM(units=params['hidden_units'], name= "lstm1",
                                   kernel_initializer='glorot_normal',return_sequences=False))(mask)
        lstm1 = Dropout(params['dropout'])(lstm1)  
        out = Dense(1,activation="sigmoid")(lstm1) 

        model = keras.models.Model(inputs=[input1, input2], outputs=out)
        model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

        return model
    
    def run_models(X_train, y_train, X_test, model,params=params ,savepath=None):
        cw = get_class_weights(X_train, y_train)
        if model == "LSTM":
            clf = build_model(X_train,params)

            es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
            checkpoint = ModelCheckpoint(filepath=savepath, monitor='val_f1',save_weights_only=False,
                                 verbose=1, save_best_only=True, mode='max')
            history = clf.fit([X_train[:,:,:18],X_train[:,:,18:]], y_train, batch_size=BATCH_SIZE,#validation_split=0.2,
                                class_weight=cw, epochs=params['epochs'],verbose=1,shuffle=True)#,
            clf.save(savepath)
            clf = load_model(savepath, custom_objects={'f1': f1})

            probs = clf.predict([X_test[:,:,:18],X_test[:,:,18:]])

        else:
            X_train_base = np.reshape(X_train,(X_train.shape[0],-1))
            y_train_base =np.expand_dims(y_train,axis=1)            
            X_test_base = np.reshape(X_test,(X_test.shape[0],-1))

            if model == "LR":
                clf = LogisticRegression(random_state=0, solver='liblinear',
                                     class_weight=cw).fit(X_train_base, y_train_base)
            elif model == "RF":
                clf = RandomForestClassifier(n_estimators=300, max_depth=6,random_state=36, max_features=8,class_weight=cw)\
                .fit(X_train_base, y_train_base)
            else:
                print('Invalid model name')

            probs = clf.predict_proba(X_test_base)[:, 1] 

        return probs
    
    def model_cv(X,y,model):
        i = 1
        cv_scores = {}

        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED_VALUE)
        for train, test in kfold.split(X, y):
            print(f'Fold: {i}')
            X_tr = X[train]
            X_ts = X[test]
            y_tr = y[train]
            y_ts = y[test]
            if high_recall:
                savepath = "mimic_%s_%d_cv_recall.hdf5"%(TASK,i)
            else:
                savepath = "mimic_%s_%d_cv.hdf5"%(TASK,i)
            probs = run_models(X_tr, y_tr, X_ts, model,params,savepath)
            cv_scores[i] = compute_metrics(X_ts,y_ts, probs)
            i += 1        

        cum_scores = avg_metrics(cv_scores)
        plot_auc(cum_scores)
        return cum_scores
    
    models = ['LR','RF','LSTM']
    results = {}
    for m in models:
        print(f'**********Model: {m}*********')
        results[m] = model_cv(X_test, y_test, model=m)

            
        if high_recall:
            file_name = '{}min_{}skip_model_{}_mimic_recall_CI_pr.json'.format(min_time,skip_time,m)
        else:
            file_name = '{}min_{}skip_model_{}_mimic_CI_pr.json'.format(min_time,skip_time,m)
        reported_results = []
        
        for r in ['mean_auc', 'ppv', 'npv', 'mcc', 'spec@90','mean_auc_prc','rec_single','l_CI','h_CI','p_l_CI','p_h_CI','r_l_CI','r_h_CI','auprc_l_CI','auprc_h_CI']:

            reported_results.append(np.round_(results[m][r],decimals=4))
            with open(file_name, 'w') as f:
                f.write(str(reported_results))
                
        No_pt = X_test.shape[0]
        pos =  y_test[y_test ==1].shape[0]
        neg =  y_test[y_test ==0].shape[0]
        with open(file_name, 'a') as f:
            
            f.write(str("Total no:"))
            f.write(str(No_pt))
            f.write(str("positive no:"))
            f.write(str(pos))
            f.write(str("negative no:"))
            f.write(str(neg))
            
        ###########################
    mean_fpr = np.linspace(0,1,100)
    fpr_tprs = [(mean_fpr, results[m]['tpr']) for m in models]
    if high_recall:
        plot_cum_auc(fpr_tprs, models, savename='{}_cv_mimic_recall_CI_pr.png'.format(TASK),auprc=False)
    else:
        plot_cum_auc(fpr_tprs, models, savename='{}_cv_mimic_CI_pr.png'.format(TASK),auprc=False)
        
    thr = sum(y_test)/len(y_test)
    
    mean_prc_rec = [(np.linspace(0,1,100), results[m]['mean_prc']) for m in models]
    if high_recall:
        plot_cum_auc(mean_prc_rec, models, savename='{}_cv_prc_mimic_recall_CI_pr.png'.format(TASK), x_label='Recall', y_label='Precision',auprc=True,thr=thr)
    else:
        plot_cum_auc(mean_prc_rec, models, savename='{}_cv_prc_mimic_CI_pr.png'.format(TASK), x_label='Recall', y_label='Precision',auprc=True,thr=thr)
        


In [ ]:
# higher recall
for m in [48,24,12]:#min time
    for s in [96,72,48,24,12]: #pred time
        get_mimic_result(m,s,high_recall=True)

In [ ]:
for m in [12]:#min time
    for s in [96,72,48,24,12]: #pred time
        get_mimic_result(m,s,high_recall=False)